This notebook helps us remove all regions were a particular degradation indicator is present from a given raster. For our particular use case, we needed to remove the 'Distorted' regions from all our orthomosaic for satellite processing.

In [1]:
import os
import rasterio
import json
from rasterio import mask
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
from rasterio import merge

In [2]:
def read_tif(path):
    dataset = rasterio.open(path)
    return dataset

def find_and_read_geojson(folder):
    file = os.path.join(folder, 'Distorted Image.geojson')
    output = read_geojson(file)
    return output

def remove_geojsons_from_base_tif(base, geojson_address):
    # convert to shape file
    gdf = gpd.read_file(geojson_address)
    gdf.to_file('del_this.shp')
    
    # read shape file and apply mask transformation
    with fiona.open("del_this.shp", "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    out_image, out_transform = mask.mask(base, shapes, invert=True)
    out_meta = base.meta
    out_meta.update({"transform": out_transform})

    return out_image, out_meta

def save_tif(save_loc, out_image, out_meta):
    with rasterio.open(save_loc, "w", **out_meta) as dest:
        dest.write(out_image)

In [3]:
base_address = '/Users/ishannangia/Desktop/TfW/tifs and labels'

Run for all below sites

In [4]:
sites = ["burntSite", "restorationSite", "chromoStrobeSite", "plantationSite"]

for site in sites:  # go into different sites

    # read a particular tiff
    site_tif_path = os.path.join(base_address, site + '.tif')
    site_tif = read_tif(site_tif_path)
    
    # remove all areas using a particular geojson file
    site_folder = os.path.join(base_address, site + '_tifs')
    geojson_file = os.path.join(site_folder, 'Distorted Image.geojson')
    new_site_image, new_site_meta = remove_geojsons_from_base_tif(site_tif, geojson_file)
    
    # save new tiff
    save_tif(os.path.join(base_address, f"all_dis_{site}.tif"), new_site_image, new_site_meta)

For Predicted DIs

Here, the second block of code is similar to the above code. However, the first block of code creates a new tif file by combining different tifs. This is used in the second block of code. So run this first one only once.

In [111]:
# merge tifs together: if you already have this tif created then no need to run it again

# defining the tifs of the sites you want to join
sites = ["burntSite", "restorationSite", "chromoStrobeSite", "plantationSite", 'dryDeciduousSite']

# collecting all tif files
base_address = '/Users/ishannangia/Desktop/TfW/tifs and labels'
site_tifs = []
for site in sites:
    site_tifs.append(rasterio.open(os.path.join(base_address, "new_" + site + '.tif')))
    
# creating new merged tif with meta dictionary
output_array, output_transform = merge.merge(site_tifs)
out_meta = site_tifs[0].meta
out_meta.update({"transform": output_transform, 'width':output_array.shape[2], 'height':output_array.shape[1]})

# saving new tif
save_tif(os.path.join(base_address, f"new_all_tifs_combined.tif"), output_array, out_meta)

In [121]:
# read a particular tiff
base_tif_path = '/Users/ishannangia/Desktop/TfW/tifs and labels/tifs_combined_wo_true_degradation.tif'
tif = read_tif(base_tif_path)

# read a particular geojson
base_address = '/Users/ishannangia/Desktop/three_dis_labelme_files'
geojson_file = os.path.join(base_address, 'Distorted Image.geojson')

# remove all vectors from the tif
new_image, new_meta = remove_geojsons_from_base_tif(tif, geojson_file)

# save new tif
save_tif(os.path.join(base_address, f"three_true_and_pred_dis_base_raster.tif"), new_image, new_meta)